In [2]:
import sys
import cv2
import glob
import os
import numpy as np
import pickle 
from random import randint
import matplotlib.pyplot as plt
from pathlib import Path
from time import time
from deep_sort import nn_matching
import project_config as config
%matplotlib inline 


In [3]:
from cav.detection import ObjectDetector
import cv2

od = ObjectDetector(config.MODEL_PATH)

2025-10-13 04:40:47.995211: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-13 04:40:48.778732: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-13 04:40:48.780840: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-13 04:40:50.769896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OpError: HoopStats_assets/models/frcnn/variables/variables.data-00000-of-00001; No such file or directory

In [4]:
CUT_UPPER = 0
SAVE_DETECTIONS = os.path.join(config.DATA_PATH, 'detections.p')
VIDEO_FILE = pickle.load(open(f'{config.DATA_PATH}/videopath.p', 'rb'))

# Create folders
Path(os.sep.join(SAVE_DETECTIONS.split(os.sep)[:-1])).mkdir(parents=True, exist_ok=True)


In [ ]:
cap = cv2.VideoCapture(VIDEO_FILE)

objects = []

results = []
colors = {}


save_detections = {}    

i = 0
t = time()

while cap.isOpened():
    video_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    video_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - CUT_UPPER     
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_timeStamp = i/fps

    t2 = time() - t
    sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
        i, t2, i/t2))                   
    i += 1
    
    ret, image = cap.read()
    if image is None:
        break

    if CUT_UPPER > 0:
        image = image[CUT_UPPER:, :, :]
    
    boxes, scores, classes = od.detect(image, timestamp=frame_timeStamp)
    if SAVE_DETECTIONS is not None:
        save_detections[i] = (boxes, scores, classes)
    
cap.release()

if SAVE_DETECTIONS is not None:
    pickle.dump(save_detections, open(SAVE_DETECTIONS,'wb'))
    print (f'\nDetections saved in {SAVE_DETECTIONS}.')

